In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('oes_pres_votes_2021.csv')

def irv(df):
    '''
    Finds the winner using an instant runoff voting system

    Parameters
    ----------
    df: pandas data frame of voting results

    Returns
    -------
    votes: tuple list of (votes, candidate) in the
        case of a tie
    sorted(votes)[-1][1]: string of the name of the
        winner
    sorted(first_choice_votes)[-1][1]: string of the
        name of the winner
    '''

    df = df.fillna('no')
    number_of_voters = len(df)

    # creating a list of candidates
    li = []
    for c in range(1, len(df.columns)+1):
        li += list(df[f'Choice{c}'].values)
    candidates = set(li)
    eliminated_candidates = []
    if 'no' in candidates:
        candidates.remove('no')

    # looping through to redistribute votes
    for round_ in range(1, len(candidates) + 1):
        voter_id = 0

        # dictionary of candidate:votes
        votes_dictionary = {}
        for cand in candidates:
            votes_dictionary[cand] = 0

        # looping through first choice votes
        for vot in list(df[f'Choice1'].values):
            i = 0

            # while the choice candidate is in the eliminated list
            while vot in eliminated_candidates:
                if i < len(df.columns):

                    # move to next choice candidate
                    vot = df.iloc[voter_id][f'Choice{i + 1}']
                else:

                    # all choice candidates have been eliminated
                    vot = 'no'
                i += 1
            if vot != 'no':
                # adding the vote to the dictionary
                votes_dictionary[vot] += 1
            voter_id += 1

        # printing vote information
        print(f"Round {round_}:")
        for cand in candidates:
            percentage_of_votes = (votes_dictionary[cand]/number_of_voters)*100
            if percentage_of_votes != 0:
                print(f"{round(percentage_of_votes, 2)}% {cand} \
({votes_dictionary[cand]} votes)")
        print()

        # creating a tuple list of (votes, candidate)
        # (easier to sort through than a dictionary)
        votes = []
        for cand in candidates:
            if cand not in eliminated_candidates:
                votes.append((votes_dictionary[cand], cand))

        # check if the most amount of votes > 50%
        if sorted(votes)[-1][0] > (.5*number_of_voters):
            return sorted(votes)[-1][1]

        # eliminating the candidate with the fewest votes
        elif len(votes) > 1:
            minimum_number_of_votes = sorted(votes)[0][0]
            maximum_number_of_votes = sorted(votes)[-1][0]
            if minimum_number_of_votes == maximum_number_of_votes:
                first_choice_votes = []
                for vote_cand in sorted(votes):
                    first_choice_votes.append((df['Choice1'].value_counts()\
                                               [vote_cand[1]], vote_cand[0]))
                if sorted(first_choice_votes)[0] == sorted(first_choice_votes)[-1]:
                    return votes
                else:
                    return sorted(first_choice_votes)[-1][1]
            for vot in votes:
                if vot[0] == minimum_number_of_votes:
                    eliminated_candidates.append(vot[1])

w = irv(df)
if isinstance(w, list):
    print('TIE between ', end = '')
    if len(w) == 2:
        print(f"{w[0][1]} and {w[1][1]}", end='')
    else:
        for wi in w:
            if wi != w[-1]:
                print(f"{wi[1]}, ", end='')
            else:
                print(f"and {wi[1]}", end='')
else:
    print(f"\n{w} is the winner.")

NameError: name 'df' is not defined